In [66]:
!pip install pymysql
pip install cryptography

SyntaxError: invalid syntax (130150470.py, line 2)

In [1]:
import socket
#장고 DB와 연동하기 위한 DATABASES 나중에 SETTINGS 에 기입해주면 된다고함
try :
    DB_IP = socket.gethostbyname('EZPZ_DB')
except:
    DB_IP = 'localhost'
USER='root'
NAME='ezpz'
ENGINE= "django.db.backends.mysql" #얘네들 따로 account때처럼 빼는건지 물어보기
DATABASES = {
    "default": {
        "ENGINE": ENGINE,
        "NAME": NAME, #나중에 db이름 고치기 데이터베이스 이름 
        "USER": USER, 
        "PASSWORD": "1234",
        "HOST": DB_IP,
        "PORT": "33306"
        
    }
}

In [2]:
import pymysql
import cryptography

In [3]:
 conn = pymysql.connect(host=DB_IP, user=USER, password='1234',db= NAME ,charset='utf8')

In [4]:
cur = conn.cursor()

In [5]:

# 테이블 일괄 삭제
cur.execute("drop table if exists comp_news;")
cur.execute("drop table if exists recruit_info;")
cur.execute("drop table if exists comp_info;")


0

In [6]:
# 기업 정보 테이블
sql = '''
    create table comp_info(comp_uid bigint auto_increment,
	comp_name varchar(20) not null,
	comp_loc varchar(50) not null,
	# 아래는 nullable
	comp_thumb varchar(2000),
    comp_cont varchar(30),
	comp_founded char(6),
	comp_size varchar(10), # 회사 규모 : 대기업, 중견기업 등
	comp_url varchar(2000),
	is_reged char(1) default 'N' not null
	check (is_reged in ('Y', 'N')),
	
    create_date char(8) not null, # DB 입력 날짜
	modify_date char(8) not null, # 최종 수정 날짜
	
	primary key comp_info(comp_uid),
	constraint COMP_NAME_UK unique comp_info(comp_name)
)
'''

# SQL 문을 실행합니다.
cur.execute(sql)

0

In [7]:
# 채용 공고 테이블
sql2='''
create table recruit_info(
	recruit_uid bigint auto_increment,
	comp_uid bigint not null, # indexing 문제로 unqiue 제약조건 삭제
	recruit_url varchar(2000) not null,
	recruit_position varchar(100) not null, #20인가에서 100으로 변경
	recruit_thumb varchar(2000),
	
	create_date char(8) not null, # DB 입력 날짜
	modify_date char(8) not null, # 최종 수정 날짜
	
	primary key recruit_info(recruit_uid),
	constraint RECRUIT_COMP_FK foreign key recruit_info(comp_uid) references comp_info(comp_uid)
)
'''

# SQL 문을 실행합니다.
cur.execute(sql2)

0

In [8]:
# 기업 뉴스 테이블
sql3='''
create table comp_news(
	news_uid bigint auto_increment,
	comp_uid bigint not null,
	pub_date char(8) not null, # 뉴스 발행 일자
	news_url varchar(2000) not null,
	news_cont varchar(5000) not null,
	
	news_sum varchar(256), # 뉴스 요약
	news_senti int(1) check (news_senti in (0, 1, 2)),
	
	create_date char(8) not null, # DB 입력 날짜
	modify_date char(8) not null, # 최종 수정 날짜
	
	primary key comp_news(news_uid),
	constraint NEWS_COMP_FK foreign key comp_news(comp_uid) references comp_info(comp_uid)
)
'''
# SQL 문을 실행합니다.
cur.execute(sql3)

0

In [9]:
cur.execute('desc comp_info')
for i in cur:
    print(i)


('comp_uid', 'bigint', 'NO', 'PRI', None, 'auto_increment')
('comp_name', 'varchar(20)', 'NO', 'UNI', None, '')
('comp_loc', 'varchar(50)', 'NO', '', None, '')
('comp_thumb', 'varchar(2000)', 'YES', '', None, '')
('comp_cont', 'varchar(30)', 'YES', '', None, '')
('comp_founded', 'char(6)', 'YES', '', None, '')
('comp_size', 'varchar(10)', 'YES', '', None, '')
('comp_url', 'varchar(2000)', 'YES', '', None, '')
('is_reged', 'char(1)', 'NO', '', 'N', '')
('create_date', 'char(8)', 'NO', '', None, '')
('modify_date', 'char(8)', 'NO', '', None, '')


In [10]:
cur.execute('desc recruit_info')
for i in cur:
    print(i)


('recruit_uid', 'bigint', 'NO', 'PRI', None, 'auto_increment')
('comp_uid', 'bigint', 'NO', 'MUL', None, '')
('recruit_url', 'varchar(2000)', 'NO', '', None, '')
('recruit_position', 'varchar(100)', 'NO', '', None, '')
('recruit_thumb', 'varchar(2000)', 'YES', '', None, '')
('create_date', 'char(8)', 'NO', '', None, '')
('modify_date', 'char(8)', 'NO', '', None, '')


In [11]:
cur.execute('desc comp_news')
for i in cur:
    print(i)


('news_uid', 'bigint', 'NO', 'PRI', None, 'auto_increment')
('comp_uid', 'bigint', 'NO', 'MUL', None, '')
('pub_date', 'char(8)', 'NO', '', None, '')
('news_url', 'varchar(2000)', 'NO', '', None, '')
('news_cont', 'varchar(5000)', 'NO', '', None, '')
('news_sum', 'varchar(256)', 'YES', '', None, '')
('news_senti', 'int', 'YES', '', None, '')
('create_date', 'char(8)', 'NO', '', None, '')
('modify_date', 'char(8)', 'NO', '', None, '')


___
## 테이블 생성완료 후 기업정보 데이터 저장하는 sql문 생성

In [12]:
import sys
sys.path.append('/app/EZPZ_Project/modules/crawlers/comp_info') #기업 정보 크롤러 경로

In [13]:
import info_crawler #기업 정보 crawler 호출

In [14]:
comp_info_df=info_crawler.get_url('트위니') # 기업정보 크롤러 모듈 객체 나중에 함수화

In [15]:
comp_info_df

,comp_uid,comp_name,comp_loc,comp_thumb,comp_cont,comp_founded,comp_size,comp_url
,321024,(주)트위니,대전 유성구 가정북로 90,https://jpassets.jobplanet.co.kr/production/up...,[솔루션/SI/ERP/CRM],2015.09.01,중소기업,http://www.twinny.ai


In [16]:
# 기업 정보 리스트로 저장해서 데이터프레임에 넣어줄수 있게
col_value=[]
for i in comp_info_df.iloc[0]:
    col_value.append(i)
print(col_value)    


['321024', '(주)트위니', '대전 유성구 가정북로 90', 'https://jpassets.jobplanet.co.kr/production/uploads/company/logo/321024/thumb_twinny-logo-1.png', ['솔루션/SI/ERP/CRM'], '2015.09.01', '중소기업', 'http://www.twinny.ai']


In [17]:
col_value[5]=col_value[5][0:7]
col_value[5] = col_value[5].replace(".", "")
col_value[5] # 6글자 문자열로 변환 테이블에 형식대로

'201509'

In [18]:
#크롤링해온 값 테이블에 저장 저장일자와 수정일자는 스케줄링단계에서 진행이므로 일단 000000 넣어두었음
cur.execute(f"INSERT INTO {NAME}.comp_info (comp_name, comp_loc, comp_thumb, comp_cont, comp_founded, comp_size, comp_url, is_reged, create_date, modify_date) VALUES (%s, %s, %s, %s, %s, %s, %s, 'N', '000000', '000000')", (col_value[1], col_value[2], col_value[3], col_value[4], col_value[5], col_value[6], col_value[7]))

1

In [19]:
cur.execute('select * from comp_info')
for i in cur:
    print(i)

(1, '(주)트위니', '대전 유성구 가정북로 90', 'https://jpassets.jobplanet.co.kr/production/uploads/company/logo/321024/thumb_twinny-logo-1.png', '솔루션/SI/ERP/CRM', '201509', '중소기업', 'http://www.twinny.ai', 'N', '000000', '000000')


In [20]:
# 작업을 커밋
conn.commit()

# 연결 종료
conn.close()

# 연결종료 후엔 반드시 다시 연결해줘야됨

___
## 조건들 더 붙여서(ex)뭐 테이블에 있는 경우(함수1)는 그냥 바로 테이블 값 select 테이블에 없는 경우는 크롤링함수호출(함수2)하는 위처럼 해서 )위 행동 모듈화